In [ ]:
pip install twython

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: /opt/anaconda3/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: /opt/anaconda3/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install vaderSentiment

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: /opt/anaconda3/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Extracting Language Features

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/ayushisabnis/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

## Data Preparation

In [ ]:
# Set the local folder path
folder_path = r'VetTrain_Transcripts'

# Function to extract the numerical part from the filename
def extract_pid(filename):
    base_name = os.path.splitext(filename)[0]
    return base_name.split('_')[0]  # Assuming filename is like "P001_transcript.csv"

# Get all CSV files in the folder and sort them numerically by filename
all_files = sorted(
    [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')],
    key=lambda x: int(extract_pid(os.path.basename(x))[1:])
)

In [ ]:
# Initialize the final storage for combined question pairs
combined_data = []

# Process each file
for file_path in all_files:
    # Extract PID from filename
    pid = extract_pid(os.path.basename(file_path))

    # Read the CSV file
    df = pd.read_csv(file_path)
    df = df[df['Type'] != 'IRR']  # Filter irrelevant rows

    # Initialize dialogue extraction
    current_dialogue = []
    current_question_id = None
    qid_counter = 1  # Start QID counter for each file

    # Extract question pairs with PID and QID
    for _, row in df.iterrows():
        if row['Type'].startswith('Q'):
            question_id = row['Type']
            if current_question_id is None:
                current_question_id = question_id
                current_dialogue = [row['Transcript']]
            elif question_id != current_question_id:
                combined_data.append({
                    'PID': pid,
                    'QID': f"Q{qid_counter}",
                    'Combined_Transcript': " ".join(current_dialogue)
                })
                qid_counter += 1
                current_dialogue = [row['Transcript']]
                current_question_id = question_id
            else:
                current_dialogue.append(row['Transcript'])
        else:
            current_dialogue.append(row['Transcript'])

    # Add the last dialogue for the file
    if current_dialogue:
        combined_data.append({
            'PID': pid,
            'QID': f"Q{qid_counter}",
            'Combined_Transcript': " ".join(current_dialogue)
        })

In [ ]:
# Convert to a DataFrame
df_combined = pd.DataFrame(combined_data)

In [ ]:
# Load the behavioral annotation codes
behavior_file = r'Behavioral Annotation Codes.csv'
df_behavior = pd.read_csv(behavior_file)

# Merge behavioral codes
df_combined = df_combined.merge(df_behavior, on=['PID', 'QID'], how='left')

In [ ]:
# Data preprocessing
import re

def clean_text(text):
    # lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Delete redundant Spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df_combined['Cleaned_Transcript'] = df_combined['Combined_Transcript'].apply(clean_text)

In [ ]:
# displaying the combined dataframe
df_combined.head(10)

,PID,QID,Combined_Transcript,Degree of Explanation,Cleaned_Transcript
0,P001,Q1,"Interviewer: Good. I just uh, I uh, always hav...",Succinct,interviewer good i just uh i uh always have it...
1,P001,Q2,"Interviewer: Okay, good deal. Interviewer: Yep...",Succinct,interviewer okay good deal interviewer yep i u...
2,P001,Q3,"Interviewer: Okay, so with all of your experie...",Under-explained,interviewer okay so with all of your experienc...
3,P001,Q4,"Interviewer: So, was that an easy transition f...",Under-explained,interviewer so was that an easy transition fro...
4,P001,Q5,Interviewer: So when we're talking about stren...,Succinct,interviewer so when were talking about strengt...
5,P001,Q6,"Interviewer: Yeah, yeah, no, I get it, I get i...",Succinct,interviewer yeah yeah no i get it i get it so ...
6,P001,Q7,Interviewer: So how did you- Interviewer: What...,Comprehensive,interviewer so how did you interviewer what wh...
7,P001,Q8,"Interviewer: Okay. Good. So, are you and your ...",Succinct,interviewer okay good so are you and your wife...
8,P001,Q9,"Interviewer: Sure. Yeah. So, when, ah, when ar...",Over-explained,interviewer sure yeah so when ah when are you ...
9,P001,Q10,"Interviewer: Yeah, I think that that is justif...",Succinct,interviewer yeah i think that that is justifie...


In [ ]:
df_combined.shape

(281, 5)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk import pos_tag, word_tokenize
import nltk

# Download required NLTK data
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ayushisabnis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ayushisabnis/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
# Count Vectorizer
vectorizer = CountVectorizer()
count_vectorizer = vectorizer.fit_transform(df_combined['Cleaned_Transcript'])
print("Feature names:", vectorizer.get_feature_names_out())
print(count_vectorizer.toarray())
np.shape(count_vectorizer.toarray())

Feature names: ['01' '0ne' '10' ... 'zone' 'zoom' 'zooming']
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


(281, 5604)

In [ ]:
# TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_features = tfidf_vectorizer.fit_transform(df_combined['Cleaned_Transcript'])
print(tfidf_features.toarray())
np.shape(tfidf_features.toarray())

[[0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 ...
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.0499286 ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]]


(281, 5604)

## Adding the extracted features to the main dataframe

In [ ]:
def add_tfidf_features(df, text_column, max_features=500):
    """
    Adds TF-IDF features to the dataset.

    Parameters:
        df (pd.DataFrame): Input dataframe containing the text data.
        text_column (str): Name of the column containing text data.
        max_features (int): Maximum number of TF-IDF features to generate (default=500).

    Returns:
        pd.DataFrame: Dataframe with TF-IDF features added.
    """
    # Initialize TF-IDF Vectorizer
    tfidf = TfidfVectorizer(max_features=max_features)

    # Fit and transform the text data
    tfidf_matrix = tfidf.fit_transform(df[text_column])

    # Convert the TF-IDF matrix to a DataFrame
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf.get_feature_names_out(), index=df.index)

    # Concatenate the TF-IDF features with the original DataFrame
    df_with_tfidf = pd.concat([df, tfidf_df], axis=1)

    return df_with_tfidf

In [ ]:
# Example usage
df_combined = add_tfidf_features(df_combined, text_column="Cleaned_Transcript", max_features=1000)
df_combined.head()

,PID,QID,Combined_Transcript,Degree of Explanation,Cleaned_Transcript,15,20,ability,able,about,...,yet,you,youd,youll,young,your,youre,yourself,youve,zoom
0,P001,Q1,"Interviewer: Good. I just uh, I uh, always hav...",Succinct,interviewer good i just uh i uh always have it...,0.0,0.0,0.0,0.0,0.086689,...,0.0,0.158225,0.0,0.0,0.0,0.000000,0.034997,0.062375,0.0,0.0
1,P001,Q2,"Interviewer: Okay, good deal. Interviewer: Yep...",Succinct,interviewer okay good deal interviewer yep i u...,0.0,0.0,0.0,0.0,0.136634,...,0.0,0.155865,0.0,0.0,0.0,0.293292,0.000000,0.000000,0.0,0.0
2,P001,Q3,"Interviewer: Okay, so with all of your experie...",Under-explained,interviewer okay so with all of your experienc...,0.0,0.0,0.0,0.0,0.073692,...,0.0,0.151316,0.0,0.0,0.0,0.067793,0.044626,0.000000,0.0,0.0
3,P001,Q4,"Interviewer: So, was that an easy transition f...",Under-explained,interviewer so was that an easy transition fro...,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.105391,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
4,P001,Q5,Interviewer: So when we're talking about stren...,Succinct,interviewer so when were talking about strengt...,0.0,0.0,0.0,0.0,0.035147,...,0.0,0.360844,0.0,0.0,0.0,0.032333,0.000000,0.000000,0.0,0.0


In [ ]:
def add_pos_tags(df, text_column):
    """
    Adds POS tagging to the dataframe.

    Parameters:
        df (pd.DataFrame): Input dataframe containing text data.
        text_column (str): Name of the column containing text data.

    Returns:
        pd.DataFrame: Dataframe with added POS tags.
    """
    def pos_tags(text):
        tokens = word_tokenize(text)
        tags = pos_tag(tokens)
        return {tag: len([word for word, pos in tags if pos == tag]) for tag in set([pos for _, pos in tags])}

    df['POS_Tags'] = df[text_column].apply(pos_tags)
    return df

In [ ]:
df_combined = add_pos_tags(df_combined, text_column="Cleaned_Transcript")
df_combined.head(10)

,PID,QID,Combined_Transcript,Degree of Explanation,Cleaned_Transcript,15,20,ability,able,about,...,you,youd,youll,young,your,youre,yourself,youve,zoom,POS_Tags
0,P001,Q1,"Interviewer: Good. I just uh, I uh, always hav...",Succinct,interviewer good i just uh i uh always have it...,0.0,0.0,0.0,0.000000,0.086689,...,0.158225,0.0,0.0,0.000000,0.000000,0.034997,0.062375,0.000000,0.0,"{'CD': 3, 'RP': 3, 'PRP$': 5, 'JJ': 36, 'CC': ..."
1,P001,Q2,"Interviewer: Okay, good deal. Interviewer: Yep...",Succinct,interviewer okay good deal interviewer yep i u...,0.0,0.0,0.0,0.000000,0.136634,...,0.155865,0.0,0.0,0.000000,0.293292,0.000000,0.000000,0.000000,0.0,"{'CD': 3, 'PRP$': 8, 'WRB': 1, 'JJ': 18, 'CC':..."
2,P001,Q3,"Interviewer: Okay, so with all of your experie...",Under-explained,interviewer okay so with all of your experienc...,0.0,0.0,0.0,0.000000,0.073692,...,0.151316,0.0,0.0,0.000000,0.067793,0.044626,0.000000,0.000000,0.0,"{'RP': 2, 'PRP$': 7, 'WRB': 5, 'JJ': 17, 'CC':..."
3,P001,Q4,"Interviewer: So, was that an easy transition f...",Under-explained,interviewer so was that an easy transition fro...,0.0,0.0,0.0,0.000000,0.000000,...,0.105391,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,"{'RP': 3, 'PRP$': 3, 'WRB': 2, 'JJ': 12, 'CC':..."
4,P001,Q5,Interviewer: So when we're talking about stren...,Succinct,interviewer so when were talking about strengt...,0.0,0.0,0.0,0.000000,0.035147,...,0.360844,0.0,0.0,0.000000,0.032333,0.000000,0.000000,0.000000,0.0,"{'PRP$': 4, 'WRB': 3, 'JJ': 18, 'CC': 9, 'PRP'..."
5,P001,Q6,"Interviewer: Yeah, yeah, no, I get it, I get i...",Succinct,interviewer yeah yeah no i get it i get it so ...,0.0,0.0,0.0,0.000000,0.000000,...,0.091637,0.0,0.0,0.000000,0.082111,0.054051,0.000000,0.000000,0.0,"{'RP': 1, 'PRP$': 5, 'WRB': 1, 'JJ': 10, 'RBS'..."
6,P001,Q7,Interviewer: So how did you- Interviewer: What...,Comprehensive,interviewer so how did you interviewer what wh...,0.0,0.0,0.0,0.000000,0.057876,...,0.198066,0.0,0.0,0.000000,0.026622,0.070096,0.000000,0.000000,0.0,"{'CD': 3, 'PRP$': 4, 'WRB': 3, 'JJ': 26, 'CC':..."
7,P001,Q8,"Interviewer: Okay. Good. So, are you and your ...",Succinct,interviewer okay good so are you and your wife...,0.0,0.0,0.0,0.085033,0.000000,...,0.035994,0.0,0.0,0.000000,0.048379,0.000000,0.000000,0.000000,0.0,"{'VBP': 4, 'NNS': 4, 'VBG': 2, 'RP': 1, 'PRP$'..."
8,P001,Q9,"Interviewer: Sure. Yeah. So, when, ah, when ar...",Over-explained,interviewer sure yeah so when ah when are you ...,0.0,0.0,0.0,0.000000,0.000000,...,0.233472,0.0,0.0,0.000000,0.044829,0.029509,0.000000,0.042184,0.0,"{'CD': 6, 'PRP$': 8, 'WRB': 10, 'JJ': 33, 'CC'..."
9,P001,Q10,"Interviewer: Yeah, I think that that is justif...",Succinct,interviewer yeah i think that that is justifie...,0.0,0.0,0.0,0.000000,0.038699,...,0.079463,0.0,0.0,0.103462,0.035601,0.046870,0.000000,0.000000,0.0,"{'PRP$': 12, 'WRB': 2, 'JJ': 16, 'CC': 9, 'PRP..."


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def add_sentiment_scores(df, text_column):
    """
    Adds sentiment scores as separate columns to the dataframe.

    Parameters:
        df (pd.DataFrame): Input dataframe containing text data.
        text_column (str): Name of the column containing text data.

    Returns:
        pd.DataFrame: Dataframe with added sentiment scores as separate columns.
    """
    sia = SentimentIntensityAnalyzer()
    sentiment_scores = df[text_column].apply(lambda text: sia.polarity_scores(text))

    # Create separate columns for each sentiment score
    df['Sentiment_Neg'] = sentiment_scores.apply(lambda score: score['neg'])
    df['Sentiment_Neu'] = sentiment_scores.apply(lambda score: score['neu'])
    df['Sentiment_Pos'] = sentiment_scores.apply(lambda score: score['pos'])
    df['Sentiment_Compound'] = sentiment_scores.apply(lambda score: score['compound'])

    return df

In [ ]:
# Adding sentiment scores as separate columns to the dataframe
df_combined = add_sentiment_scores(df_combined, text_column="Cleaned_Transcript")

# Example output for the first row
print(df_combined[['Sentiment_Neg', 'Sentiment_Neu', 'Sentiment_Pos', 'Sentiment_Compound']].iloc[0])

Sentiment_Neg         0.0070
Sentiment_Neu         0.8210
Sentiment_Pos         0.1720
Sentiment_Compound    0.9936
Name: 0, dtype: float64


In [ ]:
df_combined.drop(['QID', 'Combined_Transcript'], axis=1, inplace=True)
df_combined.head()

,PID,Degree of Explanation,Cleaned_Transcript,15,20,ability,able,about,above,absolutely,...,your,youre,yourself,youve,zoom,POS_Tags,Sentiment_Neg,Sentiment_Neu,Sentiment_Pos,Sentiment_Compound
0,P001,Succinct,interviewer good i just uh i uh always have it...,0.0,0.0,0.0,0.0,0.086689,0.0,0.0,...,0.000000,0.034997,0.062375,0.0,0.0,"{'CD': 3, 'RP': 3, 'PRP$': 5, 'JJ': 36, 'CC': ...",0.007,0.821,0.172,0.9936
1,P001,Succinct,interviewer okay good deal interviewer yep i u...,0.0,0.0,0.0,0.0,0.136634,0.0,0.0,...,0.293292,0.000000,0.000000,0.0,0.0,"{'CD': 3, 'PRP$': 8, 'WRB': 1, 'JJ': 18, 'CC':...",0.036,0.919,0.045,-0.0889
2,P001,Under-explained,interviewer okay so with all of your experienc...,0.0,0.0,0.0,0.0,0.073692,0.0,0.0,...,0.067793,0.044626,0.000000,0.0,0.0,"{'RP': 2, 'PRP$': 7, 'WRB': 5, 'JJ': 17, 'CC':...",0.011,0.904,0.085,0.9286
3,P001,Under-explained,interviewer so was that an easy transition fro...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,"{'RP': 3, 'PRP$': 3, 'WRB': 2, 'JJ': 12, 'CC':...",0.036,0.910,0.054,0.5568
4,P001,Succinct,interviewer so when were talking about strengt...,0.0,0.0,0.0,0.0,0.035147,0.0,0.0,...,0.032333,0.000000,0.000000,0.0,0.0,"{'PRP$': 4, 'WRB': 3, 'JJ': 18, 'CC': 9, 'PRP'...",0.028,0.828,0.144,0.9653


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

def add_word_embeddings(df, text_column, model_name='all-MiniLM-L6-v2'):
    """
    Adds word embeddings to the dataframe using Sentence Transformers.

    Parameters:
        df (pd.DataFrame): Input dataframe containing text data.
        text_column (str): Name of the column containing text data.
        model_name (str): Sentence Transformer model name.

    Returns:
        pd.DataFrame: Dataframe with added embeddings.
    """
    model = SentenceTransformer(model_name)
    embeddings = model.encode(df[text_column].tolist(), show_progress_bar=True)
    df['Embeddings'] = list(embeddings)
    return df

In [ ]:
df_combined = add_word_embeddings(df_combined, text_column="Cleaned_Transcript")
df_combined.head()

Batches:   0%|          | 0/9 [00:00<?, ?it/s]

,PID,Degree of Explanation,Cleaned_Transcript,15,20,ability,able,about,above,absolutely,...,youre,yourself,youve,zoom,POS_Tags,Sentiment_Neg,Sentiment_Neu,Sentiment_Pos,Sentiment_Compound,Embeddings
0,P001,Succinct,interviewer good i just uh i uh always have it...,0.0,0.0,0.0,0.0,0.086689,0.0,0.0,...,0.034997,0.062375,0.0,0.0,"{'CD': 3, 'RP': 3, 'PRP$': 5, 'JJ': 36, 'CC': ...",0.007,0.821,0.172,0.9936,"[-0.08247564, 0.009922197, 0.033036184, 0.0038..."
1,P001,Succinct,interviewer okay good deal interviewer yep i u...,0.0,0.0,0.0,0.0,0.136634,0.0,0.0,...,0.000000,0.000000,0.0,0.0,"{'CD': 3, 'PRP$': 8, 'WRB': 1, 'JJ': 18, 'CC':...",0.036,0.919,0.045,-0.0889,"[-0.029192025, 0.056575313, 0.07875369, 0.0209..."
2,P001,Under-explained,interviewer okay so with all of your experienc...,0.0,0.0,0.0,0.0,0.073692,0.0,0.0,...,0.044626,0.000000,0.0,0.0,"{'RP': 2, 'PRP$': 7, 'WRB': 5, 'JJ': 17, 'CC':...",0.011,0.904,0.085,0.9286,"[-0.042129382, 0.026093017, 0.02012703, 0.0270..."
3,P001,Under-explained,interviewer so was that an easy transition fro...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,"{'RP': 3, 'PRP$': 3, 'WRB': 2, 'JJ': 12, 'CC':...",0.036,0.910,0.054,0.5568,"[-0.06791862, 0.02170143, 0.033392277, -0.0060..."
4,P001,Succinct,interviewer so when were talking about strengt...,0.0,0.0,0.0,0.0,0.035147,0.0,0.0,...,0.000000,0.000000,0.0,0.0,"{'PRP$': 4, 'WRB': 3, 'JJ': 18, 'CC': 9, 'PRP'...",0.028,0.828,0.144,0.9653,"[0.01770237, 0.050047472, 0.011827877, -0.0141..."


# Classifying between Under-Explained and Succinct

## Cleaning Dataset

In [ ]:
# creating a dataset for classification
df_new = df_combined.copy()
df_new.head()

,PID,Degree of Explanation,Cleaned_Transcript,15,20,ability,able,about,above,absolutely,...,youre,yourself,youve,zoom,POS_Tags,Sentiment_Neg,Sentiment_Neu,Sentiment_Pos,Sentiment_Compound,Embeddings
0,P001,Succinct,interviewer good i just uh i uh always have it...,0.0,0.0,0.0,0.0,0.086689,0.0,0.0,...,0.034997,0.062375,0.0,0.0,"{'CD': 3, 'RP': 3, 'PRP$': 5, 'JJ': 36, 'CC': ...",0.007,0.821,0.172,0.9936,"[-0.08247564, 0.009922197, 0.033036184, 0.0038..."
1,P001,Succinct,interviewer okay good deal interviewer yep i u...,0.0,0.0,0.0,0.0,0.136634,0.0,0.0,...,0.000000,0.000000,0.0,0.0,"{'CD': 3, 'PRP$': 8, 'WRB': 1, 'JJ': 18, 'CC':...",0.036,0.919,0.045,-0.0889,"[-0.029192025, 0.056575313, 0.07875369, 0.0209..."
2,P001,Under-explained,interviewer okay so with all of your experienc...,0.0,0.0,0.0,0.0,0.073692,0.0,0.0,...,0.044626,0.000000,0.0,0.0,"{'RP': 2, 'PRP$': 7, 'WRB': 5, 'JJ': 17, 'CC':...",0.011,0.904,0.085,0.9286,"[-0.042129382, 0.026093017, 0.02012703, 0.0270..."
3,P001,Under-explained,interviewer so was that an easy transition fro...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,"{'RP': 3, 'PRP$': 3, 'WRB': 2, 'JJ': 12, 'CC':...",0.036,0.910,0.054,0.5568,"[-0.06791862, 0.02170143, 0.033392277, -0.0060..."
4,P001,Succinct,interviewer so when were talking about strengt...,0.0,0.0,0.0,0.0,0.035147,0.0,0.0,...,0.000000,0.000000,0.0,0.0,"{'PRP$': 4, 'WRB': 3, 'JJ': 18, 'CC': 9, 'PRP'...",0.028,0.828,0.144,0.9653,"[0.01770237, 0.050047472, 0.011827877, -0.0141..."


In [ ]:
df_new = df_new[df_new['Degree of Explanation'].isin(['Under-explained', 'Succinct'])]
df_new['DOE_Label'] = df_new['Degree of Explanation'].map({'Under-explained': 0, 'Succinct': 1})
df_new.head()

,PID,Degree of Explanation,Cleaned_Transcript,15,20,ability,able,about,above,absolutely,...,yourself,youve,zoom,POS_Tags,Sentiment_Neg,Sentiment_Neu,Sentiment_Pos,Sentiment_Compound,Embeddings,DOE_Label
0,P001,Succinct,interviewer good i just uh i uh always have it...,0.0,0.0,0.0,0.0,0.086689,0.0,0.0,...,0.062375,0.0,0.0,"{'CD': 3, 'RP': 3, 'PRP$': 5, 'JJ': 36, 'CC': ...",0.007,0.821,0.172,0.9936,"[-0.08247564, 0.009922197, 0.033036184, 0.0038...",1
1,P001,Succinct,interviewer okay good deal interviewer yep i u...,0.0,0.0,0.0,0.0,0.136634,0.0,0.0,...,0.000000,0.0,0.0,"{'CD': 3, 'PRP$': 8, 'WRB': 1, 'JJ': 18, 'CC':...",0.036,0.919,0.045,-0.0889,"[-0.029192025, 0.056575313, 0.07875369, 0.0209...",1
2,P001,Under-explained,interviewer okay so with all of your experienc...,0.0,0.0,0.0,0.0,0.073692,0.0,0.0,...,0.000000,0.0,0.0,"{'RP': 2, 'PRP$': 7, 'WRB': 5, 'JJ': 17, 'CC':...",0.011,0.904,0.085,0.9286,"[-0.042129382, 0.026093017, 0.02012703, 0.0270...",0
3,P001,Under-explained,interviewer so was that an easy transition fro...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,"{'RP': 3, 'PRP$': 3, 'WRB': 2, 'JJ': 12, 'CC':...",0.036,0.910,0.054,0.5568,"[-0.06791862, 0.02170143, 0.033392277, -0.0060...",0
4,P001,Succinct,interviewer so when were talking about strengt...,0.0,0.0,0.0,0.0,0.035147,0.0,0.0,...,0.000000,0.0,0.0,"{'PRP$': 4, 'WRB': 3, 'JJ': 18, 'CC': 9, 'PRP'...",0.028,0.828,0.144,0.9653,"[0.01770237, 0.050047472, 0.011827877, -0.0141...",1


In [ ]:
df_new.drop(['Degree of Explanation'], axis=1, inplace=True)

In [ ]:
df_new.shape

(129, 1009)

In [ ]:
# checking for null values
df_new.isna().sum()

PID                   0
Cleaned_Transcript    0
15                    0
20                    0
ability               0
                     ..
Sentiment_Neu         0
Sentiment_Pos         0
Sentiment_Compound    0
Embeddings            0
DOE_Label             0
Length: 1009, dtype: int64

In [ ]:
pos_tags_df = pd.json_normalize(df_new['POS_Tags'])
pos_tags_df.fillna(0, inplace=True)  # Replace NaN with 0

In [ ]:
pos_tags_df.shape

(129, 33)

In [ ]:
# checking for Null values
pos_tags_df.isna().sum().sum()

0

In [ ]:
# Check indices of both DataFrames
print(df_new.index)
print(pos_tags_df.index)

Index([  0,   1,   2,   3,   4,   5,   7,   9,  10,  11,
       ...
       251, 260, 264, 265, 266, 273, 275, 278, 279, 280],
      dtype='int64', length=129)
RangeIndex(start=0, stop=129, step=1)


In [ ]:
# Reset indices before concatenation
df_new = df_new.reset_index(drop=True)
pos_tags_df = pos_tags_df.reset_index(drop=True)

In [ ]:
df_new = pd.concat([df_new, pos_tags_df], axis=1)
df_new.drop(columns=['POS_Tags'], inplace=True)

In [ ]:
df_new.isna().sum()

PID                   0
Cleaned_Transcript    0
15                    0
20                    0
ability               0
                     ..
UH                    0
NNP                   0
''                    0
FW                    0
POS                   0
Length: 1041, dtype: int64

In order to filter the extracted features, we also need to handle the `Embeddings` column in such a way that each value corresponds to a single feature column.

In [ ]:
# Expand Embeddings list into individual columns
embeddings_df = pd.DataFrame(df_new['Embeddings'].to_list(), index=df_new.index)
embeddings_df.columns = [f'Embedding_{i}' for i in range(embeddings_df.shape[1])]

In [ ]:
embeddings_df.isna().sum()

Embedding_0      0
Embedding_1      0
Embedding_2      0
Embedding_3      0
Embedding_4      0
                ..
Embedding_379    0
Embedding_380    0
Embedding_381    0
Embedding_382    0
Embedding_383    0
Length: 384, dtype: int64

In [ ]:
df_new = pd.concat([df_new, embeddings_df], axis=1)
df_new.drop(columns=['Embeddings'], inplace=True)
df_new.head()

,PID,Cleaned_Transcript,15,20,ability,able,about,above,absolutely,access,...,Embedding_374,Embedding_375,Embedding_376,Embedding_377,Embedding_378,Embedding_379,Embedding_380,Embedding_381,Embedding_382,Embedding_383
0,P001,interviewer good i just uh i uh always have it...,0.0,0.0,0.0,0.0,0.086689,0.0,0.0,0.0,...,0.016676,-0.014764,0.008555,0.033396,-0.043216,0.002666,0.038463,0.003999,-0.043158,0.039700
1,P001,interviewer okay good deal interviewer yep i u...,0.0,0.0,0.0,0.0,0.136634,0.0,0.0,0.0,...,-0.115100,-0.059945,-0.027655,0.005605,-0.002344,-0.084586,0.067140,-0.086758,-0.083039,0.033452
2,P001,interviewer okay so with all of your experienc...,0.0,0.0,0.0,0.0,0.073692,0.0,0.0,0.0,...,-0.044904,0.088308,0.045410,0.036618,-0.042384,-0.000323,0.050070,0.018606,-0.082964,0.005994
3,P001,interviewer so was that an easy transition fro...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,-0.013460,-0.025747,0.005615,0.049796,-0.101872,0.001147,0.017912,-0.049540,-0.002439,0.002425
4,P001,interviewer so when were talking about strengt...,0.0,0.0,0.0,0.0,0.035147,0.0,0.0,0.0,...,0.048642,0.073907,0.025127,0.080058,0.021308,0.004080,0.038495,0.015446,-0.098640,-0.013695


In [ ]:
df_new.shape

(129, 1424)

In [ ]:
# creating a list of feature columns
feature_columns = (
    ['Sentiment_Neg', 'Sentiment_Pos', 'Sentiment_Neu', 'Sentiment_Compound'] +
    pos_tags_df.columns.tolist() +
    embeddings_df.columns.tolist()
)

In [ ]:
from sklearn.feature_selection import mutual_info_classif

def select_features_with_mutual_info(df, feature_columns, target_column, k_values):
    """
    Selects top features for multiple values of k based on mutual information.

    Parameters:
        df (pd.DataFrame): Input dataframe.
        feature_columns (list): List of feature column names.
        target_column (str): Name of the target column.
        k_values (list): List of different k values to experiment with.

    Returns:
        dict: Dictionary where keys are k values and values are lists of top k features.
    """
    X = df[feature_columns]
    y = df[target_column]

    # Calculate mutual information scores once
    mi_scores = mutual_info_classif(X, y, random_state=42)
    feature_scores = pd.Series(mi_scores, index=feature_columns)

    # Sort features by mutual information scores in descending order
    sorted_features = feature_scores.sort_values(ascending=False).index.tolist()

    # Generate top k features for all k values
    results = {k: sorted_features[:k] for k in k_values}

    return results

In [ ]:
filtered_features = select_features_with_mutual_info(
    df_new,
    feature_columns=feature_columns,
    target_column='DOE_Label',
    k_values=[100, 200, 400, 800]
)

In [ ]:
# Print results for each k
for k, features in filtered_features.items():
    print(f"Top {k} features: {features} \n")

Top 100 features: ['Embedding_332', 'WRB', 'Embedding_331', 'Embedding_230', 'Embedding_383', 'Embedding_330', 'Embedding_155', 'Embedding_133', 'Embedding_97', 'Embedding_88', 'UH', 'Embedding_114', 'Embedding_1', 'Embedding_247', 'Embedding_62', 'Embedding_316', 'Embedding_12', 'Embedding_30', 'Embedding_336', 'WDT', 'Embedding_52', 'IN', 'Embedding_44', 'Embedding_128', 'Embedding_364', 'Embedding_94', 'Embedding_182', 'Embedding_170', 'Embedding_76', 'Embedding_314', 'Embedding_131', 'Embedding_68', 'JJ', 'Embedding_226', 'Embedding_272', 'Embedding_166', 'Embedding_80', 'DT', 'Embedding_57', 'Embedding_278', 'Embedding_0', 'Embedding_359', 'Embedding_255', 'Embedding_96', 'Embedding_312', 'Embedding_346', 'Embedding_129', 'Embedding_49', 'Embedding_291', 'Embedding_210', 'Embedding_106', 'Embedding_228', 'NN', 'Embedding_305', 'Embedding_85', 'Embedding_117', 'Embedding_242', 'Embedding_2', 'Embedding_199', 'Embedding_327', 'Embedding_92', 'Embedding_326', 'Embedding_202', 'Embedd

## Splitting the Dataset

In [ ]:
from sklearn.model_selection import GroupKFold

def participant_independent_split(df, feature_columns, target_column, group_column, n_splits=5):
    """
    Splits the dataset into participant-independent folds using GroupKFold.

    Parameters:
        df (pd.DataFrame): The input dataset containing features, target, and group columns.
        feature_columns (list): List of column names to be used as features.
        target_column (str): Name of the target column.
        group_column (str): Name of the column used for grouping (e.g., Participant ID).
        n_splits (int): Number of folds (default is 5).

    Returns:
        list of tuples: Each tuple contains (X_train, X_test, y_train, y_test) for one fold.
    """
    # Extract features, target, and groups
    X = df[feature_columns]
    y = df[target_column]
    groups = df[group_column]

    # Initialize GroupKFold
    gkf = GroupKFold(n_splits=n_splits)

    # Store splits
    splits = []
    for train_idx, test_idx in gkf.split(X, y, groups):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        splits.append((X_train, X_test, y_train, y_test))

    return splits

In [ ]:
target_column = 'DOE_Label'  # Replace with the actual target column name
group_column = 'PID'  # Replace with the participant ID column name

# Call the function
folds = participant_independent_split(df_new, feature_columns, target_column, group_column, n_splits=5)

# Display train and test sets for each fold
for i, (X_train, X_test, y_train, y_test) in enumerate(folds):
    print(f"Fold {i+1}")
    print("Train PIDs:", df_new[group_column].iloc[X_train.index].unique())
    print("Test PIDs:", df_new[group_column].iloc[X_test.index].unique())
    print("-" * 50)

Fold 1
Train PIDs: ['P001' 'P002' 'P004' 'P006' 'P007' 'P010' 'P011' 'P012' 'P013' 'P014'
 'P016' 'P017' 'P018' 'P019' 'P023' 'P024' 'P025' 'P027' 'P028' 'P030'
 'P031' 'P032' 'P033' 'P038' 'P039' 'P041']
Test PIDs: ['P003' 'P005' 'P008' 'P009' 'P029' 'P036']
--------------------------------------------------
Fold 2
Train PIDs: ['P001' 'P002' 'P003' 'P004' 'P005' 'P006' 'P007' 'P008' 'P009' 'P010'
 'P011' 'P012' 'P014' 'P016' 'P019' 'P023' 'P027' 'P029' 'P030' 'P031'
 'P032' 'P033' 'P036' 'P038' 'P039' 'P041']
Test PIDs: ['P013' 'P017' 'P018' 'P024' 'P025' 'P028']
--------------------------------------------------
Fold 3
Train PIDs: ['P002' 'P003' 'P005' 'P006' 'P007' 'P008' 'P009' 'P010' 'P011' 'P012'
 'P013' 'P014' 'P016' 'P017' 'P018' 'P019' 'P023' 'P024' 'P025' 'P028'
 'P029' 'P031' 'P033' 'P036' 'P039']
Test PIDs: ['P001' 'P004' 'P027' 'P030' 'P032' 'P038' 'P041']
--------------------------------------------------
Fold 4
Train PIDs: ['P001' 'P003' 'P004' 'P005' 'P006' 'P007' 'P008

In [ ]:
# check the number of samples in each fold
for i, (X_train, X_test, y_train, y_test) in enumerate(folds):
    print(f"Fold {i+1}:")
    print(f"  Train sample shapes: {X_train.shape}")
    print(f"  Test samples shapes: {X_test.shape}")
    print("-" * 50)

Fold 1:
  Train sample shapes: (103, 421)
  Test samples shapes: (26, 421)
--------------------------------------------------
Fold 2:
  Train sample shapes: (103, 421)
  Test samples shapes: (26, 421)
--------------------------------------------------
Fold 3:
  Train sample shapes: (103, 421)
  Test samples shapes: (26, 421)
--------------------------------------------------
Fold 4:
  Train sample shapes: (103, 421)
  Test samples shapes: (26, 421)
--------------------------------------------------
Fold 5:
  Train sample shapes: (104, 421)
  Test samples shapes: (25, 421)
--------------------------------------------------


## Running Tree-Based ML Models

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, balanced_accuracy_score

def evaluate_tree_models(folds, k_features, feature_columns, target_column, top_features_dict):
    """
    Evaluates three tree-based models (Decision Tree, Random Forest, Gradient Boosting)
    with hyperparameter tuning for each k value and identifies the best model.

    Parameters:
        folds (list of tuples): Participant-independent splits (X_train, X_test, y_train, y_test).
        k_features (list): List of k values representing the number of top features to use.
        feature_columns (list): List of feature column names.
        target_column (str): Name of the target column.
        top_features_dict (dict): Dictionary mapping each k value to the list of top features.

    Returns:
        dict: Results for each k value containing the best model and its performance metrics.
    """
    # Define the models and their hyperparameter grids
    models = {
        "Decision Tree": {
            "model": DecisionTreeClassifier(random_state=42),
            "param_grid": {"max_depth": [3, 5, 10, None], "min_samples_split": [2, 5, 10]},
        },
        "Random Forest": {
            "model": RandomForestClassifier(random_state=42),
            "param_grid": {
                "n_estimators": [50, 100, 200],
                "max_depth": [3, 5, 10, None],
                "min_samples_split": [2, 5, 10],
            },
        },
        "Gradient Boosting": {
            "model": GradientBoostingClassifier(random_state=42),
            "param_grid": {
                "n_estimators": [50, 100, 200],
                "learning_rate": [0.01, 0.1, 0.2],
                "max_depth": [3, 5, 10],
            },
        },
    }

    results = {}

    # Iterate over each k value
    for k in k_features:
        print(f"Evaluating models for top {k} features...")
        top_features = top_features_dict[k]  # Get the top k features

        best_model_name = None
        best_model = None
        best_accuracy = 0
        best_balanced_accuracy = 0
        best_params = None

        # Iterate over models
        for model_name, model_info in models.items():
            print(f"Training {model_name}...")
            total_accuracy = 0
            total_balanced_accuracy = 0

            # Iterate over folds
            for X_train, X_test, y_train, y_test in folds:
                # Filter top k features for the current fold
                X_train_k = X_train[top_features]
                X_test_k = X_test[top_features]

                # Perform grid search
                grid_search = GridSearchCV(
                    model_info["model"],
                    model_info["param_grid"],
                    scoring="accuracy",
                    cv=3,
                    n_jobs=-1,
                )
                grid_search.fit(X_train_k, y_train)

                # Evaluate on test set
                best_estimator = grid_search.best_estimator_
                y_pred = best_estimator.predict(X_test_k)
                accuracy = accuracy_score(y_test, y_pred)
                balanced_accuracy = balanced_accuracy_score(y_test, y_pred)

                # Accumulate scores
                total_accuracy += accuracy
                total_balanced_accuracy += balanced_accuracy

            # Average scores over all folds
            avg_accuracy = total_accuracy / len(folds)
            avg_balanced_accuracy = total_balanced_accuracy / len(folds)

            print(f"{model_name}: Accuracy={avg_accuracy:.4f}, Balanced Accuracy={avg_balanced_accuracy:.4f} \n")

            # Update best model if this model performs better
            if (avg_accuracy > best_accuracy) and (avg_balanced_accuracy > best_balanced_accuracy):
                best_model_name = model_name
                best_model = grid_search.best_estimator_
                best_accuracy = avg_accuracy
                best_balanced_accuracy = avg_balanced_accuracy
                best_params = grid_search.best_params_

        # Store results for this k value
        results[k] = {
            "Best Model": best_model_name,
            "Best Accuracy": best_accuracy,
            "Best Balanced Accuracy": best_balanced_accuracy,
            "Best Parameters": best_params,
            "Best Model Object": best_model,
        }

    return results

In [ ]:
k_values_list = list(filtered_features.keys())
print(k_values_list)

[100, 200, 400, 800]


In [ ]:
# Example call
results = evaluate_tree_models(
    folds=folds,
    k_features=k_values_list,
    feature_columns=feature_columns,
    target_column=target_column,
    top_features_dict=filtered_features
)

# Display the best model for each k
for k, result in results.items():
    print(f"Top {k} features:")
    print(f"Best Model: {result['Best Model']}")
    print(f"Best Accuracy: {result['Best Accuracy']:.4f}")
    print(f"Best Balanced Accuracy: {result['Best Balanced Accuracy']:.4f}")
    print(f"Best Parameters: {result['Best Parameters']}")
    print("\n")

Evaluating models for top 100 features...
Training Decision Tree...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Decision Tree: Accuracy=0.7132, Balanced Accuracy=0.5062 

Training Random Forest...
Random Forest: Accuracy=0.8209, Balanced Accuracy=0.5000 

Training Gradient Boosting...
Gradient Boosting: Accuracy=0.8209, Balanced Accuracy=0.5205 

Evaluating models for top 200 features...
Training Decision Tree...
Decision Tree: Accuracy=0.7283, Balanced Accuracy=0.5132 

Training Random Forest...
Random Forest: Accuracy=0.8209, Balanced Accuracy=0.5000 

Training Gradient Boosting...
Gradient Boosting: Accuracy=0.8209, Balanced Accuracy=0.5000 

Evaluating models for top 400 features...
Training Decision Tree...
Decision Tree: Accuracy=0.7514, Balanced Accuracy=0.5193 

Training Random Forest...
Random Forest: Accuracy=0.8209, Balanced Accuracy=0.5000 

Training Gradient Boosting...
Gradient Boosting: Accuracy=0.8209, Balanced Accuracy=0.5000 

Evaluating models for top 800 features...
Training Decision Tree...
Decision Tree: Accuracy=0.7209, Balanced Accuracy=0.5315 

Training Random Forest...


In [ ]:
import joblib

# Iterate over results to save the best model for each k
for k, result in results.items():
    best_model = result["Best Model Object"]  # Get the best model object
    filename = f"partc_best_model_top_{k}_features.pkl"  # Define a filename
    joblib.dump(best_model, filename)  # Save the model
    print(f"Saved best model for top {k} features as '{filename}'.")

Saved best model for top 100 features as 'partc_best_model_top_100_features.pkl'.
Saved best model for top 200 features as 'partc_best_model_top_200_features.pkl'.
Saved best model for top 400 features as 'partc_best_model_top_400_features.pkl'.
Saved best model for top 800 features as 'partc_best_model_top_800_features.pkl'.


## Running Deep Learning Models

In order to run deep learning models such as `Conv1D` and `LSTM`, we first need to scale and reshape the input dataset because these models require their inputs to be in a specific format and shape.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def prepare_data_for_dl(X, top_features):
    """
    Scales and reshapes data for Conv1D and LSTM models.

    Parameters:
        X (pd.DataFrame): Input features.
        top_features (list): List of selected top features.

    Returns:
        np.ndarray: Scaled and reshaped data.
    """
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X[top_features])

    # Reshape for Conv1D and LSTM: (samples, timesteps, features)
    X_reshaped = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)
    return X_reshaped

The above function will be used before training and evaluating the models on each participant-independent fold.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, LSTM
from sklearn.metrics import accuracy_score, balanced_accuracy_score

def train_and_evaluate_dl_models(folds, top_features_dict, model_type="Conv1D"):
    """
    Trains and evaluates Conv1D or LSTM models on participant-independent folds without hyperparameter tuning.
    Dataset loading is handled via TensorFlow tensor slices.

    Parameters:
        folds (list of tuples): Train-test splits from GroupKFold.
        top_features_dict (dict): Dictionary of top features.
        model_type (str): "Conv1D" or "LSTM".

    Returns:
        dict: Results for each k value containing the model and its performance metrics.
    """
    results = {}

    for k, top_features in top_features_dict.items():
        print(f"Evaluating models for top {k} features...")

        best_model = None
        best_accuracy = 0
        best_balanced_accuracy = 0

        # Iterate over folds
        for fold, (X_train, X_test, y_train, y_test) in enumerate(folds):
            print(f"Training on Fold {fold + 1}...")

            # Filter dataset for top features
            X_train = X_train[top_features]
            X_test = X_test[top_features]

            # Create TensorFlow datasets
            train_ds = tf.data.Dataset.from_tensor_slices((X_train.values, y_train.values)).batch(32).prefetch(tf.data.AUTOTUNE)
            test_ds = tf.data.Dataset.from_tensor_slices((X_test.values, y_test.values)).batch(32).prefetch(tf.data.AUTOTUNE)

            # Prepare input shape
            input_shape = (len(top_features), 1)  # (timesteps, features)

            # Build model
            if model_type == "Conv1D":
                model = Sequential([
                    Conv1D(64, kernel_size=3, activation='relu', input_shape=input_shape),
                    MaxPooling1D(pool_size=2),
                    Dropout(0.2),
                    Flatten(),
                    Dense(32, activation='relu'),
                    Dense(1, activation='sigmoid')
                ])
            elif model_type == "LSTM":
                model = Sequential([
                    LSTM(64, return_sequences=True, input_shape=input_shape),
                    Dropout(0.2),
                    LSTM(32),
                    Dense(1, activation='sigmoid')
                ])
            else:
                raise ValueError("Invalid model_type. Choose 'Conv1D' or 'LSTM'.")

            # Compile model
            model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

            # Train model
            model.fit(train_ds, epochs=20, verbose=0)
            print("Evaluating on the test dataset....")

            # Evaluate on test set
            y_pred = (model.predict(test_ds) > 0.5).astype(int)
            acc = accuracy_score(y_test, y_pred)
            bal_acc = balanced_accuracy_score(y_test, y_pred)

            print(f"Fold {fold + 1}: Accuracy={acc:.4f}, Balanced Accuracy={bal_acc:.4f}\n")

            # Update best model if this model performs better
            if acc > best_accuracy and bal_acc > best_balanced_accuracy:
                best_model = model
                best_accuracy = acc
                best_balanced_accuracy = bal_acc

        # Store results for this k value
        results[k] = {
            "Best Model": best_model,
            "Best Accuracy": best_accuracy,
            "Best Balanced Accuracy": best_balanced_accuracy,
        }
        print(f"Best Model for top {k} features: Accuracy={best_accuracy:.4f}, Balanced Accuracy={best_balanced_accuracy:.4f}\n")

    return results

In [ ]:
# Call the function for Conv1D
conv1d_results = train_and_evaluate_dl_models(
    folds=folds,
    top_features_dict=filtered_features,
    model_type="Conv1D"
)

# Display results
for k, result in conv1d_results.items():
    print(f"Top {k} features:")
    print(f"Best Model: {result['Best Model']}")
    print(f"Best Accuracy: {result['Best Accuracy']:.4f}")
    print(f"Best Balanced Accuracy: {result['Best Balanced Accuracy']:.4f}\n")

Evaluating models for top 100 features...
Training on Fold 1...


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Evaluating on the test dataset....
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Fold 1: Accuracy=0.8462, Balanced Accuracy=0.7045

Training on Fold 2...


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Evaluating on the test dataset....
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Fold 2: Accuracy=0.8846, Balanced Accuracy=0.5000

Training on Fold 3...


...

Evaluating on the test dataset....
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Fold 3: Accuracy=0.7692, Balanced Accuracy=0.5000

Training on Fold 4...


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Evaluating on the test dataset....
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Fold 4: Accuracy=0.8462, Balanced Accuracy=0.4783

Training on Fold 5...


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Evaluating on the test dataset....
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Fold 5: Accuracy=0.6400, Balanced Accuracy=0.4444

Best Model for top 800 features: Accuracy=0.8462, Balanced Accuracy=0.6023

Top 100 features:
Best Model: <Sequential name=sequential_40, built=True>
Best Accuracy: 0.8462
Best Balanced Accuracy: 0.7045

Top 200 features:
Best Model: <Sequential name=sequential_46, built=True>
Best Accuracy: 0.9615
Best Balanced Accuracy: 0.8333

Top 400 features:
Best Model: <Sequential name=sequential_50, built=True>
Best Accuracy: 0.8077
Best Balanced Accuracy: 0.6818

Top 800 features:
Best Model: <Sequential name=sequential_55, built=True>
Best Accuracy: 0.8462
Best Balanced Accuracy: 0.6023



In [ ]:
# Call the function for LSTM
LSTM_results = train_and_evaluate_dl_models(
    folds=folds,
    top_features_dict=filtered_features,
    model_type="LSTM"
)

# Display results
for k, result in LSTM_results.items():
    print(f"Top {k} features:")
    print(f"Best LSTM Model: {result['Best Model']}")
    print(f"Best Accuracy: {result['Best Accuracy']:.4f}")
    print(f"Best Balanced Accuracy: {result['Best Balanced Accuracy']:.4f}\n")

Evaluating models for top 100 features...
Training on Fold 1...


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Evaluating on the test dataset....
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Fold 1: Accuracy=0.8462, Balanced Accuracy=0.5000

Training on Fold 2...


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Evaluating on the test dataset....
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Fold 2: Accuracy=0.8846, Balanced Accuracy=0.5000

Training on Fold 3...


...

Evaluating on the test dataset....
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
Fold 3: Accuracy=0.7692, Balanced Accuracy=0.5000

Training on Fold 4...


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Evaluating on the test dataset....
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
Fold 4: Accuracy=0.8846, Balanced Accuracy=0.5000

Training on Fold 5...


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Evaluating on the test dataset....
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
Fold 5: Accuracy=0.7200, Balanced Accuracy=0.5000

Best Model for top 800 features: Accuracy=0.8462, Balanced Accuracy=0.5000

Top 100 features:
Best LSTM Model: <Sequential name=sequential_60, built=True>
Best Accuracy: 0.8462
Best Balanced Accuracy: 0.5000

Top 200 features:
Best LSTM Model: <Sequential name=sequential_65, built=True>
Best Accuracy: 0.8462
Best Balanced Accuracy: 0.5000

Top 400 features:
Best LSTM Model: <Sequential name=sequential_70, built=True>
Best Accuracy: 0.8462
Best Balanced Accuracy: 0.5000

Top 800 features:
Best LSTM Model: <Sequential name=sequential_75, built=True>
Best Accuracy: 0.8462
Best Balanced Accuracy: 0.5000



In [ ]:
# Directory to save models
save_dir_conv1d = "saved_models_conv1D"
os.makedirs(save_dir_conv1d, exist_ok=True)

# Iterate through the results dictionary
for k, result in conv1d_results.items():
    best_model = result["Best Model"]
    model_path = os.path.join(save_dir_conv1d, f"partc_best_dl_conv1d_model_top_{k}_features.pkl")
    joblib.dump(best_model, model_path)  # Save the model
    print(f"Saved best model for top {k} features at: {model_path}")

Saved best model for top 100 features at: saved_models_conv1D/partc_best_dl_conv1d_model_top_100_features.pkl
Saved best model for top 200 features at: saved_models_conv1D/partc_best_dl_conv1d_model_top_200_features.pkl
Saved best model for top 400 features at: saved_models_conv1D/partc_best_dl_conv1d_model_top_400_features.pkl
Saved best model for top 800 features at: saved_models_conv1D/partc_best_dl_conv1d_model_top_800_features.pkl


In [ ]:
# Directory to save models
save_dir_lstm = "saved_models_LSTM"
os.makedirs(save_dir_lstm, exist_ok=True)

# Iterate through the results dictionary
for k, result in LSTM_results.items():
    best_model = result["Best Model"]
    model_path = os.path.join(save_dir_lstm, f"partc_best_dl_lstm_model_top_{k}_features.pkl")
    joblib.dump(best_model, model_path)  # Save the model
    print(f"Saved best model for top {k} features at: {model_path}")

Saved best model for top 100 features at: saved_models_LSTM/partc_best_dl_lstm_model_top_100_features.pkl
Saved best model for top 200 features at: saved_models_LSTM/partc_best_dl_lstm_model_top_200_features.pkl
Saved best model for top 400 features at: saved_models_LSTM/partc_best_dl_lstm_model_top_400_features.pkl
Saved best model for top 800 features at: saved_models_LSTM/partc_best_dl_lstm_model_top_800_features.pkl


In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_sample_weight

class SpeakerDebiasingPipeline:
    """
    Pipeline for reducing speaker identity dependencies in features using
    feature selection and instance reweighting.
    """

    def __init__(self, n_features_to_remove=10):
        self.n_features_to_remove = n_features_to_remove
        self.removed_features = None
        self.scaler = StandardScaler()

    def identify_speaker_dependent_features(self, X, pids, features):
        """
        Identify features most dependent on speaker identity using mutual information.

        Args:
            X: Feature matrix
            pids: Array of speaker IDs
            features: List of feature names
        """
        # Calculate mutual information between features and speaker ID
        mi_scores = mutual_info_classif(
            X[features],
            pids,
            random_state=42
        )

        # Create feature importance DataFrame
        feature_importance = pd.DataFrame({
            'feature': features,
            'mi_score': mi_scores
        })

        # Sort features by mutual information score
        feature_importance = feature_importance.sort_values(
            'mi_score',
            ascending=False
        )

        # Select top N most speaker-dependent features
        self.removed_features = feature_importance['feature'].head(
            self.n_features_to_remove
        ).tolist()

        print("Features removed due to high speaker dependency:")
        for idx, row in feature_importance.head(self.n_features_to_remove).iterrows():
            print(f"{row['feature']}: {row['mi_score']:.4f}")

        return self.removed_features

    def remove_speaker_dependent_features(self, X, features):
        """Remove identified speaker-dependent features from dataset."""
        if self.removed_features is None:
            raise ValueError("Must run identify_speaker_dependent_features first")

        remaining_features = [f for f in features if f not in self.removed_features]
        return remaining_features, X[remaining_features]

    def compute_balanced_weights(self, y, pids):
        """Compute sample weights to balance both class and protected attribute."""
        combined_labels = [f"{y_i}_{p_i}" for y_i, p_i in zip(y, pids)]
        weights = compute_sample_weight('balanced', combined_labels)
        return weights

    def evaluate_model(self, y_true, y_pred, pids):
        """Evaluate model performance with fairness metrics."""
        from sklearn.metrics import accuracy_score, balanced_accuracy_score

        metrics = {
            'accuracy': accuracy_score(y_true, y_pred),
            'balanced_accuracy': balanced_accuracy_score(y_true, y_pred)
        }

        # Calculate per-speaker accuracy
        for pid in np.unique(pids):
            mask = pids == pid
            metrics[f'accuracy_pid_{pid}'] = accuracy_score(y_true[mask], y_pred[mask])

        # Calculate demographic parity
        pred_rates = {}
        for pid in np.unique(pids):
            mask = pids == pid
            pred_rates[pid] = np.mean(y_pred[mask])

        metrics['demographic_parity_diff'] = max(pred_rates.values()) - min(pred_rates.values())

        return metrics

def run_debiased_pipeline(df_new, feature_columns, target_column, folds, models_dict):
    """
    Run complete debiasing pipeline with cross-validation.
    """
    pipeline = SpeakerDebiasingPipeline(n_features_to_remove=10)
    results = []

    for fold_idx, (X_train, X_test, y_train, y_test) in enumerate(folds):
        print(f"\nProcessing Fold {fold_idx + 1}")

        # Get PIDs for train and test sets
        # Assuming PIDs are in df_new with the same index as X_train/X_test
        pid_train = df_new.loc[X_train.index]['PID'].values
        pid_test = df_new.loc[X_test.index]['PID'].values

        # Remove speaker-dependent features
        print("\nIdentifying speaker-dependent features...")
        speaker_features = pipeline.identify_speaker_dependent_features(
            X_train, pid_train, feature_columns
        )
        remaining_features, X_train_filtered = pipeline.remove_speaker_dependent_features(
            X_train, feature_columns
        )
        _, X_test_filtered = pipeline.remove_speaker_dependent_features(
            X_test, feature_columns
        )

        # Scale features
        X_train_scaled = pipeline.scaler.fit_transform(X_train_filtered)
        X_test_scaled = pipeline.scaler.transform(X_test_filtered)

        # Compute balanced weights
        sample_weights = pipeline.compute_balanced_weights(y_train, pid_train)

        print(f"\nTraining models with {len(remaining_features)} features...")

        # Train and evaluate each model
        for model_name, model in models_dict.items():
            print(f"\nTraining {model_name}...")

            # Fit model with sample weights
            model.fit(X_train_scaled, y_train, sample_weight=sample_weights)

            # Get predictions
            y_pred = model.predict(X_test_scaled)

            # Evaluate results
            fold_metrics = pipeline.evaluate_model(y_test, y_pred, pid_test)
            fold_metrics.update({
                'fold': fold_idx,
                'model': model_name
            })
            results.append(fold_metrics)

            print(f"{model_name} Results:")
            for metric, value in fold_metrics.items():
                if metric not in ['fold', 'model']:
                    print(f"{metric}: {value:.4f}")

    return pd.DataFrame(results)

# Example usage:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

# Define models to evaluate
models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Logistic Regression': LogisticRegression(random_state=42)
}

# Run pipeline
results = run_debiased_pipeline(
    df_new=df_new,
    feature_columns=feature_columns,
    target_column='DOE_Label',
    folds=folds,
    models_dict=models
)

# Print summary results
print("\nAverage Results by Model:")
summary = results.groupby('model').mean().drop('fold', axis=1)
print(summary)

print("\nStandard Deviation of Results by Model:")
std_dev = results.groupby('model').std().drop('fold', axis=1)
print(std_dev)


Processing Fold 1

Identifying speaker-dependent features...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Features removed due to high speaker dependency:
Embedding_302: 0.3796
Embedding_364: 0.3384
Embedding_250: 0.3127
Embedding_253: 0.3118
Embedding_238: 0.3046
Embedding_222: 0.2995
Embedding_225: 0.2992
Embedding_217: 0.2959
Embedding_220: 0.2920
MD: 0.2916

Training models with 411 features...

Training Random Forest...
Random Forest Results:
accuracy: 0.8462
balanced_accuracy: 0.5000
accuracy_pid_P003: 0.7778
accuracy_pid_P005: 1.0000
accuracy_pid_P008: 0.7500
accuracy_pid_P009: 0.8000
accuracy_pid_P029: 1.0000
accuracy_pid_P036: 1.0000
demographic_parity_diff: 0.0000

Training Decision Tree...
Decision Tree Results:
accuracy: 0.7308
balanced_accuracy: 0.5341
accuracy_pid_P003: 0.6667
accuracy_pid_P005: 0.7500
accuracy_pid_P008: 1.0000
accuracy_pid_P009: 0.8000
accuracy_pid_P029: 0.5000
accuracy_pid_P036: 0.5000
demographic_parity_diff: 0.5000

Training Logistic Regression...
Logistic Regression Results:
accuracy: 0.8462
balanced_accuracy: 0.6023
accuracy_pid_P003: 0.6667
accuracy_pi

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# Load a pre-trained transformer model and tokenizer

model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Define a pipeline for zero-shot classification or text classification
classifier = pipeline("zero-shot-classification", model=model_name)

# Example labeled transcripts for few-shot learning
few_shot_examples = [
    {"text": "The explanation skipped important details about the process.", "label": "Under-explained"},
    {"text": "The answer was clear and concise.", "label": "Succinct"},
    {"text": "There were some good points, but the explanation could have been clearer.", "label": "Under-explained"},
    {"text": "The response was to the point and adequately covered the question.", "label": "Succinct"}
]

# Create a prompt with few-shot examples
def create_few_shot_prompt(transcript):
    examples_text = "\n".join([
        f"Example: \"{example['text']}\" -> Label: {example['label']}"
        for example in few_shot_examples
    ])
    prompt = (
        f"The task is to classify the transcript as 'Under-explained' or 'Succinct' based on its explanation quality.\n"
        f"Here are some examples:\n{examples_text}\n\n"
        f"Transcript: \"{transcript}\"\n"
        f"Label:"
    )
    return prompt

# Function to classify transcript and generate reasoning
def classify_and_reason(transcript):
    # Use the classifier to predict the label
    prompt = create_few_shot_prompt(transcript)
    result = classifier(
        sequences=transcript,
        candidate_labels=["Under-explained", "Succinct"],
        hypothesis_template="This text is {}."
    )

    # Extract the most likely label
    label = result["labels"][0]

    # Generate reasoning
    reasoning = f"The transcript was classified as '{label}' because it aligns with the characteristics of {label} examples, such as {'succinctness and clarity' if label == 'Succinct' else 'lack of detail and clarity'}."

    return label, reasoning

# Example usage
transcript = "The steps for solving the problem were not clearly outlined, making it hard to follow."
label, reasoning = classify_and_reason(transcript)

print("Label:", label)
print("Reasoning:", reasoning)

# Add evaluation metrics (accuracy, precision, recall)
def evaluate_model(transcripts, true_labels):
    from sklearn.metrics import accuracy_score, precision_recall_fscore_support

    predictions = [classify_and_reason(t)[0] for t in transcripts]
    accuracy = accuracy_score(true_labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='weighted')

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1
    }

# Example evaluation
# Replace with your test dataset
transcripts = [
    "The answer was to the point and addressed the query effectively.",
    "Critical details were missing, making it hard to grasp the full explanation."
]
true_labels = ["Succinct", "Under-explained"]

metrics = evaluate_model(transcripts, true_labels)
print("Evaluation Metrics:", metrics)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-MiniLM-L6-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-MiniLM-L6-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use mps:0
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


Label: Succinct
Reasoning: The transcript was classified as 'Succinct' because it aligns with the characteristics of Succinct examples, such as succinctness and clarity.
Evaluation Metrics: {'accuracy': 0.5, 'precision': 0.25, 'recall': 0.5, 'f1_score': 0.3333333333333333}


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
